In [2]:
from google.colab import files

uploaded = files.upload()

Saving sentiment_dataset.csv to sentiment_dataset.csv


In [1]:
# Run this cell first to fix the bitsandbytes issue
!pip install -U bitsandbytes
!pip install -U transformers accelerate
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.8 MB/s eta 0:00:00
PyTorch version: 2.8.0+cu126
CUDA available: False


In [3]:
# Install all required packages
!pip install -U bitsandbytes peft safetensors
!pip install transformers datasets torch accelerate gradio pandas scikit-learn
!pip install -U transformers accelerate

# Create offload directory
import os
os.makedirs("./offload", exist_ok=True)

print("✅ All packages installed and offload directory created!")

✅ All packages installed and offload directory created!


In [1]:
# Run this first to clear memory
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

!pip install transformers datasets torch accelerate gradio pandas

In [4]:
import torch
import pandas as pd
import gradio as gr
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from datasets import Dataset
import gc
import os
import warnings
import re
warnings.filterwarnings("ignore")

# Memory management
os.environ["WANDB_DISABLED"] = "true"
torch.backends.cudnn.benchmark = False

class ImprovedSentimentLLM:
    def __init__(self, model_name="microsoft/DialoGPT-small"):  # Better conversational model
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        self.trained_model = None

        # Enhanced sentiment patterns for better fallbacks
        self.sentiment_patterns = {
            'very_positive': {
                'keywords': ['amazing', 'fantastic', 'wonderful', 'excellent', 'perfect', 'outstanding', 'brilliant', 'awesome', 'incredible', 'spectacular'],
                'response': "Very Positive 🌟 You're radiating amazing energy! Keep embracing these wonderful moments and let them fuel your day!"
            },
            'positive': {
                'keywords': ['happy', 'good', 'great', 'excited', 'joy', 'pleased', 'glad', 'cheerful', 'delighted', 'optimistic', 'fun', 'yay'],
                'response': "Positive 😊 Your positive energy is contagious! It's beautiful to see you enjoying life's moments."
            },
            'very_negative': {
                'keywords': ['hate', 'terrible', 'awful', 'horrible', 'disgusting', 'furious', 'devastated', 'miserable', 'depressed'],
                'response': "Very Negative 😔 I hear you're going through something really tough. Consider reaching out to someone you trust or a professional for support."
            },
            'negative': {
                'keywords': ['sad', 'angry', 'upset', 'bad', 'frustrated', 'annoyed', 'disappointed', 'worried', 'stressed', 'down'],
                'response': "Negative 😟 It sounds like you're having a challenging time. Take a deep breath and remember that difficult feelings are temporary."
            },
            'neutral': {
                'keywords': ['okay', 'fine', 'normal', 'alright', 'so-so', 'meh', 'whatever'],
                'response': "Neutral 😐 You seem to be in a balanced emotional space. Sometimes that's exactly where we need to be."
            }
        }

    def clear_memory(self):
        """Clear GPU/CPU memory"""
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

    def setup_model_and_tokenizer(self):
        """Initialize model with 4-bit quantization for better efficiency"""
        print(f"🚀 Loading improved model ({self.model_name})...")

        # Clear memory first
        self.clear_memory()

        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        # 4-bit quantization config for memory efficiency
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
        ) if torch.cuda.is_available() else None

        # Load model with quantization
        try:
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                quantization_config=quantization_config,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                low_cpu_mem_usage=True,
                device_map="auto" if torch.cuda.is_available() else None
            )

            print("✅ Model loaded with quantization!")

        except Exception as e:
            print(f"Quantization failed: {e}")
            # Fallback to regular loading
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float32,
                low_cpu_mem_usage=True
            )
            if torch.cuda.is_available():
                self.model = self.model.cuda()
            print("✅ Model loaded (fallback mode)!")

        print(f"Model parameters: ~{sum(p.numel() for p in self.model.parameters()) / 1e6:.1f}M")

    def analyze_sentiment_patterns(self, text):
        """Analyze text using pattern matching for better fallbacks"""
        text_lower = text.lower()

        # Count sentiment indicators
        sentiment_scores = {
            'very_positive': 0,
            'positive': 0,
            'very_negative': 0,
            'negative': 0,
            'neutral': 0
        }

        for sentiment_type, data in self.sentiment_patterns.items():
            for keyword in data['keywords']:
                if keyword in text_lower:
                    sentiment_scores[sentiment_type] += 1

        # Find dominant sentiment
        max_score = max(sentiment_scores.values())
        if max_score > 0:
            dominant_sentiment = max(sentiment_scores, key=sentiment_scores.get)
            return self.sentiment_patterns[dominant_sentiment]['response']

        return "Neutral 😐 I'm processing your message. Take a moment to reflect on how you're feeling right now."

    def prepare_better_dataset(self, data_list):
        """Create better formatted dataset"""
        training_texts = []
        for item in data_list:
            # Use a more structured format
            prompt = f"Human: {item['input_text']}\nAssistant: {item['target_text']}<|endoftext|>"
            training_texts.append(prompt)
        return training_texts

    def create_optimized_dataset(self, training_texts):
        """Create optimized dataset with better examples"""
        print(f"Creating optimized dataset from {len(training_texts)} examples...")

        # Use more examples but keep them short
        training_texts = training_texts[:16]  # More examples for better learning

        # Tokenize with appropriate sequence length
        tokenized_data = []
        for text in training_texts:
            encoded = self.tokenizer(
                text,
                truncation=True,
                max_length=200,  # Slightly longer for better context
                padding='max_length',
                return_tensors='pt'
            )
            tokenized_data.append({
                'input_ids': encoded['input_ids'].squeeze().tolist(),
                'attention_mask': encoded['attention_mask'].squeeze().tolist(),
                'labels': encoded['input_ids'].squeeze().tolist()
            })

        dataset = Dataset.from_list(tokenized_data)
        print(f"✅ Optimized dataset created with {len(dataset)} examples!")
        return dataset

    def improved_fine_tune(self, training_texts):
        """Improved fine-tuning with better parameters"""
        print("🔥 Starting improved fine-tuning...")

        train_dataset = self.create_optimized_dataset(training_texts)

        # Better training arguments
        training_args = TrainingArguments(
            output_dir="./improved_model",
            num_train_epochs=2,  # More epochs
            per_device_train_batch_size=2,
            gradient_accumulation_steps=4,
            learning_rate=3e-4,  # Better learning rate
            warmup_steps=5,
            logging_steps=2,
            save_steps=100,
            save_strategy="no",
            logging_strategy="steps",
            report_to="none",
            remove_unused_columns=False,
            dataloader_num_workers=0,
            fp16=torch.cuda.is_available(),
            max_steps=20,  # More training steps
            dataloader_pin_memory=False,
            optim="adamw_torch",
        )

        data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer,
            mlm=False
        )

        try:
            trainer = Trainer(
                model=self.model,
                args=training_args,
                train_dataset=train_dataset,
                data_collator=data_collator,
            )

            print("🚀 Training...")
            trainer.train()
            print("✅ Improved fine-tuning completed!")

            self.trained_model = self.model

        except Exception as e:
            print(f"Training failed: {e}")
            print("Using base model with pattern matching...")
            self.trained_model = self.model

        self.clear_memory()

    def clean_response(self, response):
        """Clean and improve model responses"""
        # Remove repetitive patterns
        response = re.sub(r'(Sentiment:\s*)+', '', response)
        response = re.sub(r'(Advice:\s*)+', '', response)

        # Remove incomplete sentences
        sentences = response.split('.')
        cleaned_sentences = []
        for sentence in sentences:
            sentence = sentence.strip()
            if len(sentence) > 10 and sentence.count(' ') > 1:  # Filter out fragments
                cleaned_sentences.append(sentence)

        if cleaned_sentences:
            response = '. '.join(cleaned_sentences[:2])  # Max 2 sentences
            if not response.endswith('.'):
                response += '.'

        return response.strip()

    def generate_response(self, input_text):
        """Generate improved response with multiple fallback layers"""
        if self.trained_model is None:
            return "Model not ready!"

        try:
            prompt = f"Human: {input_text}\nAssistant:"

            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=150
            )

            if torch.cuda.is_available() and next(self.trained_model.parameters()).is_cuda:
                inputs = {k: v.cuda() for k, v in inputs.items()}

            with torch.no_grad():
                outputs = self.trained_model.generate(
                    **inputs,
                    max_new_tokens=60,
                    temperature=0.7,  # Lower temperature for more focused responses
                    do_sample=True,
                    top_p=0.9,
                    top_k=50,
                    pad_token_id=self.tokenizer.eos_token_id,
                    num_return_sequences=1,
                    early_stopping=True,
                    repetition_penalty=1.1
                )

            full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = full_response.replace(prompt, "").strip()

            # Clean the response
            response = self.clean_response(response)

            # If response is still poor, use pattern matching
            if (not response or
                len(response) < 10 or
                'Sentiment:' in response or
                response.count('Advice:') > 1):
                response = self.analyze_sentiment_patterns(input_text)

            return response

        except Exception as e:
            print(f"Generation error: {e}")
            return self.analyze_sentiment_patterns(input_text)

# Enhanced training data with better examples
enhanced_sample_data = [
    {"input_text": "I feel sad and depressed today", "target_text": "Negative 😢 I understand you're going through a difficult time. These feelings are valid, and it's important to reach out for support when you need it."},
    {"input_text": "I am really happy today", "target_text": "Positive 😊 That's wonderful to hear! Your happiness is contagious. Keep embracing these joyful moments."},
    {"input_text": "I'm confused about everything lately", "target_text": "Neutral 😐 Feeling confused is completely normal. Take things one step at a time and give yourself space to process your thoughts."},
    {"input_text": "This situation is absolutely terrible", "target_text": "Negative 😠 I can hear your frustration. When things feel overwhelming, try breaking down the situation into smaller, manageable parts."},
    {"input_text": "Having an amazing day with friends", "target_text": "Positive 🎉 That sounds fantastic! Good friends and positive experiences are precious. Treasure these beautiful moments."},
    {"input_text": "I don't know what to think anymore", "target_text": "Neutral 🤔 Uncertainty can be uncomfortable, but it's also an opportunity for growth. Be patient with yourself as you figure things out."},
    {"input_text": "I hate how things are going", "target_text": "Negative 😤 Strong feelings like this show that something important to you needs attention. Consider what changes might help improve your situation."},
    {"input_text": "Everything is going perfectly right now", "target_text": "Positive ✨ How wonderful! These perfect moments are gifts. Soak up this positive energy and let it strengthen you."},
    {"input_text": "yay this was fun", "target_text": "Positive 🎊 I love your enthusiasm! It's great to hear you had such a fun experience. Keep seeking out those joyful moments!"},
    {"input_text": "feeling okay I guess", "target_text": "Neutral 😐 Sometimes 'okay' is exactly where we need to be. There's no pressure to feel anything more than what's authentic for you right now."},
]

def main():
    """Main function with improved setup"""
    print("🧠 Improved Memory-Optimized Sentiment Analysis")
    print("Using DialoGPT-small (~117M parameters) with 4-bit quantization!")

    # Initialize
    llm = ImprovedSentimentLLM()
    llm.setup_model_and_tokenizer()

    # Load dataset
    try:
        print("Loading sentiment_dataset.csv...")
        df = pd.read_csv('sentiment_dataset.csv')
        print(f"✅ Found {len(df)} entries")

        custom_data = []
        for _, row in df.iterrows():
            custom_data.append({
                "input_text": str(row['input_text'])[:150],
                "target_text": str(row['target_text'])[:200]
            })

        if len(custom_data) < 5:  # If not enough custom data
            custom_data.extend(enhanced_sample_data)

        training_data = llm.prepare_better_dataset(custom_data)

    except Exception as e:
        print(f"Using enhanced sample data: {e}")
        training_data = llm.prepare_better_dataset(enhanced_sample_data)

    # Improved training
    llm.improved_fine_tune(training_data)

    # Create improved UI
    def predict(text):
        if not text.strip():
            return "Please enter some text to analyze!"

        response = llm.generate_response(text[:300])
        return response

    # Enhanced Gradio interface
    interface = gr.Interface(
        fn=predict,
        inputs=gr.Textbox(
            lines=3,
            placeholder="Share what's on your mind...",
            label="Your Message"
        ),
        outputs=gr.Textbox(label="Sentiment Analysis & Response", lines=4),
        title="💫 Enhanced Sentiment Analysis AI",
        description="Improved AI with better understanding and more natural responses!",
        examples=[
            ["yay this was so much fun today!"],
            ["I'm feeling down and confused"],
            ["Had an amazing time with my family"],
            ["This is really frustrating me"],
            ["I'm not sure how I feel right now"],
            ["Everything is going perfectly!"],
            ["I hate how complicated this is"]
        ],
        theme=gr.themes.Soft(),
        css="""
        .gradio-container {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }
        """
    )

    print("🚀 Launching enhanced interface...")
    interface.launch(
        share=True,
        debug=False,
        quiet=True
    )

if __name__ == "__main__":
    # Memory check
    if torch.cuda.is_available():
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f}GB")
        print("Using 4-bit quantization for memory efficiency!")

    main()

🧠 Improved Memory-Optimized Sentiment Analysis
Using DialoGPT-small (~117M parameters) with 4-bit quantization!
🚀 Loading improved model (microsoft/DialoGPT-small)...


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/351M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

✅ Model loaded with quantization!
Model parameters: ~124.4M
Loading sentiment_dataset.csv...
✅ Found 2003 entries
🔥 Starting improved fine-tuning...
Creating optimized dataset from 2003 examples...
✅ Optimized dataset created with 16 examples!
🚀 Training...


Step,Training Loss
2,12.555600
4,10.241500
6,5.374200
8,3.374600
10,1.858800
12,1.148000
14,0.821700
16,0.640900
18,0.535000
20,0.546200


✅ Improved fine-tuning completed!
🚀 Launching enhanced interface...
* Running on public URL: https://c46694414a109583f4.gradio.live
